In [335]:
pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [336]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import library yang diperlukan seperti :**


*   Pandas = analisis data ke bentuk dataframe/tabel
*   sklearn = melakukan TF-IDF, reduksi dimensi dan model klasifikasi



In [337]:
import pandas as pd
import string
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB

**Baca data**

In [338]:
df = pd.read_csv("dataset-baru.csv")
df

,Unnamed: 0,Abstrak,Kategori
0,0,Sistem informasi akademik (SIAKAD) merupaka...,RPL
1,1,Berjalannya koneksi jaringan komputer dengan l...,RPL
2,2,Web server adalah sebuah perangkat lunak serve...,RPL
3,3,Penjadwalan kuliah di Perguruan Tinggi me...,KOMPUTASI
4,4,Seiring perkembangan teknologi yang ada diduni...,RPL
...,...,...,...
800,804,Investasi saham selama ini memiliki resiko ker...,KOMPUTASI
801,805,Information Retrieval (IR) merupakan pengambil...,KOMPUTASI
802,806,Klasifikasi citra merupakan proses pengelompok...,KOMPUTASI
803,807,Identifikasi atribut pejalan kaki merupakan sa...,KOMPUTASI


**Melakukan pengambilan data dari setiap kolom dalam dataframe**

**Cetak data kolom kategori**

In [339]:
text = df['Abstrak']
Y = df['Kategori'].values
Y

array(['RPL', 'RPL', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI',
       'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL',
   

**Proses NLP**


---



*   case_folding = memperkecil setiap huruf
*   cleaning text = menghilangkan simbol
*   tokenisasi = memecah kalimat menjadi kata
*   stopword = menghilangkan kata yang tidak penting
*   stemming = menjadikan kata dasar





In [340]:
def case_folding(comment):
    comment = comment.lower()
    return comment

X = text.apply(case_folding)

In [341]:
symbols = ",!\"#$%&()*+-.…/:;<=>?@[\]^_`{|}~\n"
def cleaning_text(comment):
    comment = re.sub(r"\s—\s", "", comment)
    comment = re.sub(r"http\S+", "", comment)
    for i in symbols:
        comment = np.char.replace(comment, i, ' ')
    return comment
X = X.apply(cleaning_text)

In [342]:
def tokenize(comment):
    return comment.split()
X = X.apply(tokenize)

In [343]:
nltk.download('stopwords')
list_stopwords = stopwords.words('indonesian')
def stopword_removal(comment):
    return [word for word in comment if word not in list_stopwords]
X = X.apply(stopword_removal)
df['Abstrak'] = X
df['Abstrak']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0      [sistem, informasi, akademik, siakad, sistem, ...
1      [berjalannya, koneksi, jaringan, komputer, lan...
2      [web, server, perangkat, lunak, server, berfun...
3      [penjadwalan, kuliah, perguruan, kompleks, per...
4      [seiring, perkembangan, teknologi, didunia, mu...
                             ...                        
800    [investasi, saham, memiliki, resiko, kerugian,...
801    [information, retrieval, ir, pengambilan, info...
802    [klasifikasi, citra, proses, pengelompokan, pi...
803    [identifikasi, atribut, pejalan, kaki, salah, ...
804    [topik, deteksi, objek, menarik, perhatian, pe...
Name: Abstrak, Length: 805, dtype: object

In [344]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming(comment):
    kata = []
    for term in comment:
        term = stemmer.stem(term)
        kata.append(term)
    return kata
X = X.apply(stemming)
df['Abstrak'] = X
df['Abstrak']

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [jalan, koneksi, jaring, komputer, lancar, gan...
2      [web, server, perangkat, lunak, server, fungsi...
3      [jadwal, kuliah, guru, kompleks, masalah, vari...
4      [iring, kembang, teknologi, dunia, muncul, tek...
                             ...                        
800    [investasi, saham, milik, resiko, rugi, gera, ...
801    [information, retrieval, ir, ambil, informasi,...
802    [klasifikasi, citra, proses, kelompok, piksel,...
803    [identifikasi, atribut, pejal, kaki, salah, te...
804    [topik, deteksi, objek, tarik, perhati, kemban...
Name: Abstrak, Length: 805, dtype: object

**Proses TF-IDF**

---
*   Menginisialisasi objek TfidfVectorizer yang berfungsi untuk mengubah
*   Melakukan pengaplikasian TfidfVectorizer pada dokumen teks dan disimpan
*   Mengkonversi matriks TF-IDF menjadi dataframe Pandas yang merepresentasikan setiap kata pada dokumen teks yang diberi bobot berdasarkan TF-IDF
*   Tampilkan isi dari hasil konversi










In [345]:
# Inisialisasi objek TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Terapkan TfidfVectorizer pada dokumen
tfidf_matrix = tfidf_vectorizer.fit_transform(text)

# Ubah matriks TF-IDF menjadi dataframe Pandas
df_tfidf = pd.DataFrame(
    tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()
)

X = df_tfidf.values
print(df_tfidf)

      00  000  00001  0001  000505341  000506436  000792353  00086  00089  \
0    0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
1    0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
2    0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
3    0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
4    0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
..   ...  ...    ...   ...        ...        ...        ...    ...    ...   
800  0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
801  0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
802  0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
803  0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   
804  0.0  0.0    0.0   0.0        0.0        0.0        0.0    0.0    0.0   

     001  ...   zf  zona  zone  zoning  zoom  zucara  zungu   zâ   âµ   â¼ 

**Proses Pembagian Dataset**

---
Bagi data dengan data training 90% dan data test 10%



In [346]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.1, random_state = 100)

y_train

array(['KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'RPL',
       'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'RPL',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI',
       'KOMPUTASI', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL',
       'RPL', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI',

**Melatih dan mengevaluasi kinerja dari klasifikasi menggunakan algoritma Naive Bayes.**

---


*   Inisialisasi model Naive Bayes Gaussian
*   Melatih model menggunakan data training
*   Mengevaluasi kinerja model pada data test
*   Cetak akurasi






In [347]:
# Train a Naive Bayes classifier on the reduced dataset
nb = GaussianNB()
nb.fit(X_train, y_train)

# Evaluate the classifier on the test data
accuracy = nb.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 76.54%


Melakukan proses reduksi dimensi menggunakan PCA agar mempercepat proses komputasi

**Proses reduksi dimensi**

---


*   Inisialisasi objek PCA
*   Transformasi fitur-fitur dari data training dan data test
*   lihat presentase dari variansi total yang dijelaskan oleh setiap komponen utama yang digunakan untuk memastikan tidak kehilangan informasi yang signifikan dari data setelah reduksi dimensi.






In [348]:
pca = PCA(n_components =25)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

pca.explained_variance_ratio_

array([0.01903064, 0.01382074, 0.01198357, 0.00966306, 0.00849257,
       0.00794838, 0.00726448, 0.00693901, 0.00654351, 0.00637988,
       0.00628839, 0.00607725, 0.00591353, 0.00569493, 0.00540839,
       0.00531662, 0.00511611, 0.00494309, 0.00484286, 0.00475261,
       0.00457977, 0.00443965, 0.00435028, 0.00433172, 0.00426394])

**Melakukan pelatihan dan mengevaluasi ulang kinerja dari klasifikasi menggunakan algoritma Naive Bayes.**

In [349]:
# Train a Naive Bayes classifier on the reduced dataset
nb = GaussianNB()
nb.fit(X_train, y_train)

# Evaluate the classifier on the test data
accuracy = nb.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 86.42%


**Kesimpulan**

---
Klasifikasi dataset menggunakan metode naive bayes menghasilkan akurasi 76.54%

Untuk mempercepat proses komputasi maka dilakukanlah reduksi dimensi menggunakan PCA. Setelah melakukan skenario uji coba dengan n_componen(5,10,15,20,25,30), n_componen(25)-lah yang menghasilkan akurasi tertinggi dengan yaitu 86.42%
